In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

In [2]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
pic_cache_path = "E:\\workspace\\movie_center_ui\\cache\\javbus_pic_cache"
root_url = "https://www.javbus.com"
user_data_path = "E:\\workspace\\movie_center_ui\\cache\\chromium_cache_80\\user-data"
disk_cache_path = "E:\\workspace\\movie_center_ui\\cache\\chromium_cache_80\\disk-cache"
chrome_exe_path = "E:\\workspace\\movie_center_ui\\chrome-win-80\\chrome.exe"
chrome_driver_path = "E:\\workspace\\movie_center_ui\\selenium_scraper\\chromedriver_800397.exe"
proxy_server_address = "http://localhost:1080"

chrome_options=Options()
chrome_options.add_argument("--headless") #设置chrome浏览器无界面模式
chrome_options.binary_location = chrome_exe_path
chrome_options.add_argument("user-data-dir=%s" % user_data_path)
chrome_options.add_argument("disk-cache-dir=%s" % disk_cache_path)
chrome_options.add_argument("window-size=1000,1000")
chrome_options.add_argument("proxy-server=%s" % proxy_server_address)
chrome_options.add_argument("--disable-web-security")

#建立浏览器实例
browser = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver_path)
browser.set_script_timeout(120)
# 开始请求
browser.get(root_url)

In [4]:
import pymongo
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.movieTest


In [5]:
from selenium.webdriver.remote.webelement import WebElement
from IPython import display
from base64 import b64decode

def display_element(element: WebElement):
  # 预览一个web元素的缩略图
  # 只有在一个块的最后一个输出时 才会出现
  return display.Image(b64decode(element.screenshot_as_base64))
  
def get_brick_info(element: WebElement):
  photo_img_ele = element.find_element_by_xpath("//div[@class='photo-frame']/img")
  photo_img_src = photo_img_ele.get_attribute("src")
  title = photo_img_ele.get_attribute("title")
  date_tags = element.find_elements_by_tag_name("date")
  serial = date_tags[0].get_attribute("innerHTML").strip()
  date = date_tags[1].get_attribute("innerHTML").strip()
  info_page_herf = element.find_element_by_tag_name("a").get_attribute("href")
  
  return {
    'title':title,
    'date':date,
    'serial':serial,
    'info_page_href':info_page_herf,
    'photo_img_src':photo_img_src 
  }



In [6]:
# _info_tags = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p")
# 
# serial = _info_tags[0].find_element_by_xpath("./span[2]").get_attribute("innerHTML").strip()
# print(_info_tags)
# print(serial)

In [7]:
def movie_insert_update(db, browser: webdriver.Chrome, _brick):
  # 查看有没有记录
  movie_collection = db['javbus_movie']
  one = movie_collection.find_one({
    "serial": _brick['serial'],
  })
  if not one is None:
    return str(one['_id'])
 
  # 进入网页
  browser.get(_brick['info_page_href'])
  
  # 等信息加载完
  _info_tags = []
  try:
    xpath = "//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p"
    locator = (By.XPATH,xpath)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located(locator))
    _info_tags = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p")
  except Exception as e:
    print('page load fail:[%s]' % browser.current_url,e)
    return;
    
  # 开始爬取信息
  
  # 识别码 发行日期
  serial = _info_tags[0].find_element_by_xpath("./span[2]").get_attribute("innerHTML").strip()
  release_date = _info_tags[1].text.split(": ")[1].strip()
 
 # 三个带有链接的信息 信息还可能不存在
  tags = browser.find_elements_by_xpath\
    ("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p/a/preceding-sibling::*")
  tag_texts = ["導演:","製作商:","發行商:"]
  studio = {
    'name': '',
    'href': '',
  }
  lab = {
    'name': '',
    'href': '',
  }
  director = {
    'name': '',
    'href': '',
  }
  for i,tag in enumerate(tags):
    tag_text = tag.text
    a_tag = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p/a")[i]
    if tag_text == tag_texts[0]:
      # 导演
      director['name'] = a_tag.text.strip()
      director['href'] = a_tag.get_attribute("href")
    if tag_text == tag_texts[1]:
      # studio
      studio['name'] = a_tag.text.strip()
      studio['href'] = a_tag.get_attribute("href")
    if tag_text == tag_texts[2]:
      # lab
      lab['name'] = a_tag.text.strip()
      lab['href'] = a_tag.get_attribute("href")
   
  # 演员列表
  # 暂无演出者列表的时候 要特殊处理 这个时候 就没有p标签了！！
  jav_idols = []
  category_p_tag_index = -3 # 有演员列表时 类别是倒数第3个 没有时 是倒数第2个
  try:
    # 有 idol 列表时 有ul标签
    browser.find_element_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/ul")
  except NoSuchElementException:
    category_p_tag_index = -2
  else:
    jav_idol_tags = _info_tags[-1].find_elements_by_xpath("./span/a")
    jav_idols = [{
      'name': x.text.strip(),
      'href': x.get_attribute('href')
    } for x in jav_idol_tags]
    
  # 类别
  category_tags = _info_tags[category_p_tag_index].find_elements_by_xpath("./span/a")
  categories = [{
    'name': x.text.strip(),
    'href': x.get_attribute("href"),
  } for x in category_tags]
  
  # 磁力连接
  magnet_tags = browser.find_elements_by_xpath("//table[@id='magnet-table']/tr/td/a[1]")
  magnets = [
    {
    'magnet_link': magnet_tags[x].get_attribute('href'),
    'size': magnet_tags[x+1].text.strip(),
    'date': magnet_tags[x+2].text.strip(),
  } for x in range(0,len(magnet_tags),3) ]
  
  # 样品图像
  sample_tags = browser.find_elements_by_xpath("//div[@id='sample-waterfall']/a")
  samples = [{
    'name': "%s-sample-pic-%d.%s" % (serial, i,x.get_attribute("href").split('.')[-1]),
    'href': x.get_attribute('href')
  } for i,x in enumerate(sample_tags)]
  
  # 封面
  cover_href = browser.find_element_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class,'screencap')]/a/img")\
    .get_attribute("src")
  cover_pic_name = "%s-cover-pic.%s" % (serial, cover_href.split('.')[-1])
  
  # 向数据库中插入数据
  for i, id in enumerate(category_insert_update(db, categories)):
    categories[i]['id'] = id
  for i, id in enumerate(jav_idol_insert_update(db, jav_idols)):
    jav_idols[i]['id'] = id
  if studio['name'] != '':
    studio['id'] = studio_insert_update(db, studio)
  if lab['name'] != '':
    lab['id'] = lab_insert_update(db, lab)
  if director['name'] != '':
    director['id'] = director_insert_update(db, director)
  
  # 下载图片
  download_pic(cover_href, cover_pic_name)
  for sample in samples:
    download_pic(sample['href'], sample['name'])
  
  # 插入主页面
  one = movie_collection.insert_one({
    'serial':serial,
    'cover': cover_pic_name,
    'release_date': release_date,
    'studio': studio,
    'lab': lab,
    'director': director,
    'category': categories,
    'idol': jav_idols,
    'magnet': magnets,
    'sample_pic': samples,
  })
  return one.inserted_id

def category_insert_update(db, categories):
  category_collection: pymongo.collection.Collection = db['javbus_category']
  ret = []
  for category in categories:
    name = category['name']
    one = category_collection.find_one({"name": name})
    if one is None:
      ret.append(category_collection.insert_one(category).inserted_id)
    else:
      ret.append(str(one['_id']))
  return ret

def jav_idol_insert_update(db, idols):
  idol_collection = db['javbus_idol']
  ret = []
  for idol in idols:
    name = idol['name']
    one = idol_collection.find_one({"name":name})
    if one is None:
      ret.append(idol_collection.insert_one(idol).inserted_id)
    else:
      ret.append(str(one['_id']))
  return ret

def studio_insert_update(db, studio):
  studio_collection = db['javbus_studio']
  one = studio_collection.find_one({"name":studio['name']})
  if one is None:
    return studio_collection.insert_one(studio).inserted_id
  else:
    return str(one['_id'])

def lab_insert_update(db, lab):
  lab_collection = db['javbus_lab']
  one = lab_collection.find_one({"name":lab['name']})
  if one is None:
    return lab_collection.insert_one(lab).inserted_id
  else:
    return str(one['_id'])

def director_insert_update(db, director):
  director_collection = db['javbus_director']
  one = director_collection.find_one({"name": director['name']})
  if one is None:
    return director_collection.insert_one(director).inserted_id
  else:
    return str(one['_id'])
  
def download_pic(src, filename, dst_path=pic_cache_path):
  bytes_str = browser.execute_async_script("""
    var callback = arguments[arguments.length -1];
    var src = arguments[arguments.length-2];
    
    xhr = new XMLHttpRequest()
    xhr.responseType = "arraybuffer"
    xhr.open('GET',src)
    xhr.send()
    xhr.onreadystatechange = () => {
      if (xhr.readyState === XMLHttpRequest.DONE) {
        callback(new Uint8Array(xhr.response))
      }
    }
  """, src)
  raw_data = bytes(list(bytes_str))
  f = open("%s/%s" % (dst_path, filename), 'wb') # TODO 用操作系统系统的字符串拼接
  f.write(raw_data)
  f.close()
# movie_insert_update(db, browser)


In [8]:
# 向后爬取count页的信息
def scrapy_pages(count):
  for _count in range(count):
    
    # 找到页面内所有的 砖头
    _locator = (By.XPATH, "/html/body/div[@class='container-fluid']/div/div[@id='waterfall']/div/div")
    WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located(_locator))
    _bricks = browser.find_elements_by_xpath("/html/body/div[@class='container-fluid']/div/div[@id='waterfall']/div/div")
    _bricks = [get_brick_info(brick) for brick in _bricks]
    
    if len(browser.window_handles) == 1:
      # 新建窗口
      browser.execute_script("""
        window.open('about:blank','_blank');
      """)
    # 焦点转到第二个窗口上
    browser.switch_to.window(browser.window_handles[1])
    
    # 在第二个窗口上爬取详情页面
    for _brick in _bricks:
      movie_insert_update(db,browser,_brick)
      
    # 返回页码窗口
    browser.switch_to.window(browser.window_handles[0])

    # 翻页了！！ 
    _locator = (By.ID,"next")
    WebDriverWait(browser, 10).until(EC.presence_of_element_located(_locator))
    next_tag = browser.find_element_by_id("next")
    # next_tag.click()   # 很奇怪  元素不出现在可视窗口中 就不能点击
    browser.get(next_tag.get_attribute('href'))


In [ ]:

browser.switch_to_window(browser.window_handles[0])
try:
  scrapy_pages(10)
except TimeoutException as e:
  print(e.msg)
  print(e.stacktrace)




e:\workspace\movie_center_ui\selenium_scraper\venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: use driver.switch_to.window instead
  """Entry point for launching an IPython kernel.


In [23]:
# TODO
# 2. 捕捉错误 记录错误的url 重新爬取
